In [29]:
import pandas as pd

In [30]:
# set fred api key
fred_api_key = "653ed88f4bdd6c12141a8c80288845c4"
cpi_series_id = "CPIAUCSL"

In [31]:
# get cpi data
cpi = pd.read_json(f"https://api.stlouisfed.org/fred/series/observations?series_id={cpi_series_id}&api_key={fred_api_key}&file_type=json")

# make a dataframe from the observations
cpi = pd.DataFrame(cpi["observations"].values.tolist())

# select the date and value columns
cpi = cpi[["date", "value"]]
cpi.columns = ["date", "cpi"]

# convert date to datetime and cpi to numeric
cpi["date"] = pd.to_datetime(cpi["date"])
cpi['cpi'] = pd.to_numeric(cpi['cpi'])

# generate a yoy change column
cpi["cpi_yoy"] = cpi["cpi"].pct_change(12)

cpi

,date,cpi,cpi_yoy
0,1947-01-01,21.480,NaN
1,1947-02-01,21.620,NaN
2,1947-03-01,22.000,NaN
3,1947-04-01,22.000,NaN
4,1947-05-01,21.950,NaN
...,...,...,...
931,2024-08-01,314.121,0.025912
932,2024-09-01,314.686,0.024075
933,2024-10-01,315.454,0.025763
934,2024-11-01,316.441,0.027326


In [34]:
# get zori data
zori = pd.read_csv("data/zori_us.csv")

In [35]:
zori

,RegionID,SizeRank,RegionName,RegionType,StateName,date,zori
0,102001,0,United States,country,NaN,2015-01-31,1195.719167
1,102001,0,United States,country,NaN,2015-02-28,1202.219551
2,102001,0,United States,country,NaN,2015-03-31,1211.129410
3,102001,0,United States,country,NaN,2015-04-30,1220.137128
4,102001,0,United States,country,NaN,2015-05-31,1229.364590
...,...,...,...,...,...,...,...
115,102001,0,United States,country,NaN,2024-08-31,1973.698072
116,102001,0,United States,country,NaN,2024-09-30,1974.704526
117,102001,0,United States,country,NaN,2024-10-31,1970.929209
118,102001,0,United States,country,NaN,2024-11-30,1967.844967


In [28]:
zori = pd.read_csv("data/zori_us.csv")

zori

,RegionID,SizeRank,RegionName,RegionType,StateName,date,zori
0,102001,0,United States,country,NaN,2015-01-31,1195.719167
1,102001,0,United States,country,NaN,2015-02-28,1202.219551
2,102001,0,United States,country,NaN,2015-03-31,1211.129410
3,102001,0,United States,country,NaN,2015-04-30,1220.137128
4,102001,0,United States,country,NaN,2015-05-31,1229.364590
...,...,...,...,...,...,...,...
115,102001,0,United States,country,NaN,2024-08-31,1973.698072
116,102001,0,United States,country,NaN,2024-09-30,1974.704526
117,102001,0,United States,country,NaN,2024-10-31,1970.929209
118,102001,0,United States,country,NaN,2024-11-30,1967.844967


In [36]:

# restrict to cols of interest
zori = zori[["date", "zori"]]

# convert date to datetime
zori["date"] = pd.to_datetime(zori["date"])

zori

/var/folders/jd/5wy1jytx2pg8j4jr12tl0k_m0000gn/T/ipykernel_45857/823327755.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zori["date"] = pd.to_datetime(zori["date"])


,date,zori
0,2015-01-31,1195.719167
1,2015-02-28,1202.219551
2,2015-03-31,1211.129410
3,2015-04-30,1220.137128
4,2015-05-31,1229.364590
...,...,...
115,2024-08-31,1973.698072
116,2024-09-30,1974.704526
117,2024-10-31,1970.929209
118,2024-11-30,1967.844967


In [37]:
zori.dtypes

date    datetime64[ns]
zori           float64
dtype: object

In [38]:
# date is currently the last day of the month, so we need to shift it to the first day of the month
zori["date"] = zori["date"] + pd.offsets.MonthBegin(-1)
zori

/var/folders/jd/5wy1jytx2pg8j4jr12tl0k_m0000gn/T/ipykernel_45857/3726904441.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zori["date"] = zori["date"] + pd.offsets.MonthBegin(-1)


,date,zori
0,2015-01-01,1195.719167
1,2015-02-01,1202.219551
2,2015-03-01,1211.129410
3,2015-04-01,1220.137128
4,2015-05-01,1229.364590
...,...,...
115,2024-08-01,1973.698072
116,2024-09-01,1974.704526
117,2024-10-01,1970.929209
118,2024-11-01,1967.844967


In [39]:
# add yoy change column
zori["zori_yoy"] = zori["zori"].pct_change(12)
zori

/var/folders/jd/5wy1jytx2pg8j4jr12tl0k_m0000gn/T/ipykernel_45857/3867379269.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zori["zori_yoy"] = zori["zori"].pct_change(12)


,date,zori,zori_yoy
0,2015-01-01,1195.719167,NaN
1,2015-02-01,1202.219551,NaN
2,2015-03-01,1211.129410,NaN
3,2015-04-01,1220.137128,NaN
4,2015-05-01,1229.364590,NaN
...,...,...,...
115,2024-08-01,1973.698072,0.033451
116,2024-09-01,1974.704526,0.033203
117,2024-10-01,1970.929209,0.032888
118,2024-11-01,1967.844967,0.034015


In [40]:
# merge the two dataframes
# we want to keep all the dates in the zori dataframe
# so we use a left join
rent_inflation = pd.merge(
    zori,
    cpi,
    on="date",
    how="left"
    )

rent_inflation = rent_inflation[["date", "zori_yoy", "cpi_yoy"]]

rent_inflation

,date,zori_yoy,cpi_yoy
0,2015-01-01,NaN,-0.002299
1,2015-02-01,NaN,-0.000870
2,2015-03-01,NaN,-0.000220
3,2015-04-01,NaN,-0.001040
4,2015-05-01,NaN,0.000350
...,...,...,...
115,2024-08-01,0.033451,0.025912
116,2024-09-01,0.033203,0.024075
117,2024-10-01,0.032888,0.025763
118,2024-11-01,0.034015,0.027326


In [41]:
# melt the dataframe
rent_inflation = rent_inflation.melt(id_vars="date", var_name="metric", value_name="yoy_change")

In [42]:
# change the values of the metric column to zori_yoy:Rent and cpi_yoy:CPI
# make a dictionary to map the old values to the new values
metric_map = {
    "zori_yoy": "Rent",
    "cpi_yoy": "CPI"
}

# use the map method to change the values
rent_inflation["metric"] = rent_inflation["metric"].map(metric_map)

In [43]:
# save to csv
rent_inflation.to_csv("data/rent_inflation.csv", index=False)